In [25]:
import  os, glob
import  random, csv
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import pickle
import cv2
from PIL import Image
import adapt
from adapt.feature_based import DANN
from tensorflow import keras
from keras import layers
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, Dense, Reshape
from tensorflow.keras.optimizers import Adam
from keras.utils.np_utils import to_categorical  


# The path of the datasets, use dict format
dataset_path = {"base": "dataset/ccpd/splitted_plates_base", 
                "challenge":"dataset/ccpd/splitted_plates_challenge",
               "db":"dataset/ccpd/splitted_plates_db",
               "fn":"dataset/ccpd/splitted_plates_fn",
               "weather":"dataset/ccpd/splitted_plates_weather"}
save_check_pt = './checkpoints_DANN'
my_epoch = 500

def load_csv(root, filename, name2label):
    # From csv file return images dir,labels list
    if not os.path.exists(os.path.join(root, filename)):
        images = []
        for name in name2label.keys(): 
            images += glob.glob(os.path.join(root, name, '*.png'))
            images += glob.glob(os.path.join(root, name, '*.jpg'))
            images += glob.glob(os.path.join(root, name, '*.jpeg'))
        #print(len(images), images)
        random.shuffle(images) # shuffle images
        with open(os.path.join(root, filename), mode='w', newline='') as f:
            writer = csv.writer(f)
            for img in images:  
                name = img.split(os.sep)[-2]
                label = name2label[name]
                writer.writerow([img, label])
            print('written into csv file:', filename)
    # read existed csv
    images, labels = [], []
    with open(os.path.join(root, filename)) as f:
        reader = csv.reader(f)
        for row in reader:
            img, label = row
            label = int(label)
            images.append(img)
            labels.append(label) 
    # return img dir and label
    return images, labels

def load_ccpd(root, mode='train'):
    name2label = {}  # "sq...":0
    # iterate sub dir, sort, while keep mapping
    for name in sorted(os.listdir(os.path.join(root))):
        # skip non file folder
        if not os.path.isdir(os.path.join(root, name)):
            continue
        # give each category a number
        name2label[name] = len(name2label.keys())
    images, labels = load_csv(root, 'images.csv', name2label)
    if mode == 'train':  # 20%
        images = images[:int(0.2 * len(images))]
        labels = labels[:int(0.2 * len(labels))]
    elif mode == 'val':  # 20% = 60%->80%
        images = images[int(0.6 * len(images)):int(0.8 * len(images))]
        labels = labels[int(0.6 * len(labels)):int(0.8 * len(labels))]
    else:  # all
        images = images
        labels = labels
    return images, labels, name2label

# def readCcpdImg(images_dir):
#     X = []
#     for img_dir in images_dir:
#         img = cv2.imread(img_dir)
#         img = cv2.resize(img,(32,32))
#         X.append(img)
#     X = np.array(X)
#     return X

def readCcpdImg(images_dir):
    X = []
    for img_dir in images_dir:
        img = Image.open(img_dir)
        img = img.convert('L') # conver to grayscale images
        img = img.resize([32, 16])
        img_np = np.asarray(img)
        X.append(img_np.reshape([-1]))
    X = np.array(X)
    return X

def get_img_label(path, mode,num_classes=35):
    images_dir, labels, table = load_ccpd(dataset_path[path],mode=mode)
    images = readCcpdImg(images_dir)
    labels = np.array(labels)
    labels = to_categorical(labels, num_classes=num_classes)
    return images,labels

def get_encoder(input_shape=(512,)):
    model = Sequential()
    model.add(Dense(512, activation='relu',
                    input_shape=input_shape))
    model.add(Dense(256, activation="relu"))
    model.add(Dense(128, activation="sigmoid"))
    return model

def get_task(input_shape=(128,)):
    model = Sequential()
    model.add(Dense(128, activation='relu',input_shape=input_shape))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(35, activation="softmax"))
    return model

def get_discriminator(input_shape=(128,)):
    model = Sequential()
    model.add(Dense(128, activation='relu',input_shape=input_shape))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    return model

base_images,base_labels = get_img_label('base', 'all', num_classes=35)
weather_images,weather_labels = get_img_label('weather', 'all', num_classes=35)
weather_images_t,weather_labels_t = get_img_label('weather', 'train', num_classes=35)
challenge_images,challenge_labels = get_img_label('challenge', 'all', num_classes=35)
challenge_images_t,challenge_labels_t = get_img_label('challenge', 'train', num_classes=35)
db_images,db_labels = get_img_label('db', 'all', num_classes=35)
db_images_t,db_labels_t = get_img_label('db', 'train', num_classes=35)
fn_images,fn_labels = get_img_label('fn', 'all', num_classes=35)
fn_images_t,fn_labels_t = get_img_label('fn', 'train', num_classes=35)

In [23]:
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

#Dummy inputs to load model
xs=np.zeros((8446, 512))
ys=np.zeros((8446, 35))
xt=np.zeros((762, 512))
yt=np.zeros((762, 35))
weather_model = DANN(get_encoder(), get_task(), get_discriminator(),
             lambda_=1.0, optimizer=Adam(0.0001),loss='CategoricalCrossentropy',metrics=["acc"],random_state=0)
weather_model.fit(xs, ys,xt,yt, epochs=0,verbose=1,batch_size = 32) # NO fit happen, just dummy step to load model
weather_model.load_weights(save_check_pt+ '/weather')
weather_model.score(weather_images, weather_labels)
weather_hat = weather_model.predict(weather_images)
weather_hat = np.argmax(weather_hat,axis=1)
weather_labels = np.argmax(weather_labels,axis=1)
print(classification_report(weather_labels, weather_hat))

1/1 [==============================] - 0s 123ms/step - loss: 4.0621 - acc: 0.7644
              precision    recall  f1-score   support

           0       0.94      0.92      0.93       248
           1       0.84      0.93      0.88       137
           2       0.92      0.96      0.94       231
           3       0.98      0.94      0.96       172
           4       0.88      0.91      0.89        78
           5       0.99      0.89      0.93       233
           6       0.96      0.94      0.95       236
           7       0.90      0.96      0.93       213
           8       0.96      0.96      0.96       257
           9       0.92      0.96      0.94       224
          10       0.97      0.98      0.97       748
          11       1.00      0.88      0.94        50
          12       0.89      0.98      0.93        49
          13       0.74      0.90      0.81        48
          14       0.85      0.79      0.81        28
          15       0.90      0.90      0.90        40

In [27]:
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

#Dummy inputs to load model
xs=np.zeros((8446, 512))
ys=np.zeros((8446, 35))
xt=np.zeros((109, 512))
yt=np.zeros((109, 35))
challenge_model = DANN(get_encoder(), get_task(), get_discriminator(),
             lambda_=1.0, optimizer=Adam(0.0001),loss='CategoricalCrossentropy',metrics=["acc"],random_state=0)
challenge_model.fit(xs, ys,xt,yt, epochs=0,verbose=1,batch_size = 32) # NO fit happen, just dummy step to load model
challenge_model.load_weights(save_check_pt+ '/challenge')
challenge_model.score(challenge_images, challenge_labels)
challenge_hat = challenge_model.predict(challenge_images)
challenge_hat = np.argmax(challenge_hat,axis=1)
challenge_labels = np.argmax(challenge_labels,axis=1)
print(classification_report(challenge_labels, challenge_hat))

1/1 [==============================] - 0s 114ms/step - loss: 4.9436 - acc: 0.6770
              precision    recall  f1-score   support

           0       0.81      0.87      0.84        39
           1       0.78      0.66      0.71        38
           2       0.82      0.93      0.87        30
           3       0.78      0.88      0.82        16
           4       0.67      0.67      0.67         6
           5       0.88      0.86      0.87        35
           6       0.82      0.86      0.84        37
           7       0.82      0.82      0.82        28
           8       0.94      0.85      0.89        34
           9       0.97      0.67      0.79        42
          10       0.83      0.95      0.88        95
          11       0.88      0.78      0.82         9
          12       0.83      0.71      0.77         7
          13       0.80      0.44      0.57         9
          14       0.33      0.33      0.33         6
          15       0.67      1.00      0.80         8

In [28]:
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

#Dummy inputs to load model
xs=np.zeros((8446, 512))
ys=np.zeros((8446, 35))
xt=np.zeros((373, 512))
yt=np.zeros((373, 35))
db_model = DANN(get_encoder(), get_task(), get_discriminator(),
             lambda_=1.0, optimizer=Adam(0.0001),loss='CategoricalCrossentropy',metrics=["acc"],random_state=0)
db_model.fit(xs, ys,xt,yt, epochs=0,verbose=1,batch_size = 32) # NO fit happen, just dummy step to load model
db_model.load_weights(save_check_pt+ '/db')
db_model.score(db_images, db_labels)
db_hat = db_model.predict(db_images)
db_hat = np.argmax(db_hat,axis=1)
db_labels = np.argmax(db_labels,axis=1)
print(classification_report(db_labels, db_hat))

1/1 [==============================] - 0s 119ms/step - loss: 4.7142 - acc: 0.7057
              precision    recall  f1-score   support

           0       0.82      0.75      0.78       106
           1       0.48      0.78      0.60        37
           2       0.84      0.96      0.89       123
           3       0.90      0.88      0.89       121
           4       0.82      0.82      0.82        39
           5       0.85      0.83      0.84       122
           6       0.98      0.82      0.89       142
           7       0.71      0.88      0.79        75
           8       0.94      0.84      0.89       138
           9       0.94      0.65      0.77       139
          10       0.91      0.95      0.93       372
          11       0.74      0.71      0.73        28
          12       0.71      0.73      0.72        30
          13       0.30      0.43      0.35        14
          14       0.48      0.68      0.57        22
          15       0.47      0.39      0.43        23

In [29]:
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

#Dummy inputs to load model
xs=np.zeros((8446, 512))
ys=np.zeros((8446, 35))
xt=np.zeros((48, 512))
yt=np.zeros((48, 35))
fn_model = DANN(get_encoder(), get_task(), get_discriminator(),
             lambda_=1.0, optimizer=Adam(0.0001),loss='CategoricalCrossentropy',metrics=["acc"],random_state=0)
fn_model.fit(xs, ys,xt,yt, epochs=0,verbose=1,batch_size = 32) # NO fit happen, just dummy step to load model
fn_model.load_weights(save_check_pt+ '/fn')
fn_model.score(fn_images, fn_labels)
fn_hat = fn_model.predict(fn_images)
fn_hat = np.argmax(fn_hat,axis=1)
fn_labels = np.argmax(fn_labels,axis=1)
print(classification_report(fn_labels, fn_hat))

1/1 [==============================] - 0s 112ms/step - loss: 4.8389 - acc: 0.6708
              precision    recall  f1-score   support

           0       0.89      1.00      0.94        16
           1       0.75      0.30      0.43        10
           2       1.00      1.00      1.00        14
           3       0.80      1.00      0.89         8
           4       0.60      0.60      0.60         5
           5       1.00      0.80      0.89        10
           6       0.81      1.00      0.90        13
           7       0.83      0.83      0.83        12
           8       0.92      0.79      0.85        14
           9       1.00      0.80      0.89        15
          10       0.86      0.96      0.91        52
          11       0.50      1.00      0.67         1
          12       0.75      1.00      0.86         6
          13       0.33      0.25      0.29         4
          14       0.00      0.00      0.00         0
          15       0.67      1.00      0.80         2

In [ ]:
# fig = plt.figure()
# img_h, img_w = 32,16
# fig.subplots_adjust(hspace=0.4, wspace=0.4)
# for i in range(6):
#     ax = fig.add_subplot(2, 3, i+1)
#     ax.imshow(weather_images[i].reshape([img_h, img_w]))

#model.save_weights('ckpt')
#dummy step to load model from checkpoint
#xt = np.zeros((3812, 512))
#nmd = DANN(lambda_=0.1, Xt=xt, metrics=["acc"], random_state=0)
#x = np.zeros((1,1))
#y = np.zeros(1)
#nmd.fit(x,y, epochs=0,verbose=1)
#nmd.summary()
#nmd.load_weights("ckpt")
#nmd.score(weather_images, weather_labels)

#print(model.predict(weather_images[0].reshape([-1,512])))
#y0 = model.predict(weather_images[0].reshape([-1,512]))
#print(np.argmax(y0))
# y_hat = model.predict(weather_images)
# y_hat = np.argmax(y_hat,axis=1)
# weather_labels = np.argmax(weather_labels,axis=1)
# print(classification_report(weather_labels, y_hat))
# y_hat = model.predict(base_images)
# y_hat = np.argmax(y_hat,axis=1)
# base_labels = np.argmax(base_labels,axis=1)
# print(classification_report(base_labels, y_hat))